***Importamos las librerías necesarias :***

In [ ]:
import os
import torch
import zipfile
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
from io import BytesIO
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torchvision.models import alexnet, AlexNet_Weights
from torchvision.models import resnet50, ResNet50_Weights
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from torchvision.models import densenet121, DenseNet121_Weights
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights

***Guardamos las imágenes con sus respectivas clases en "complete_df" :***

In [ ]:
zip_path = './Imagenes/Trayectorias_comprimidas/longitud_28/raw-img.zip'

complete_df = pd.DataFrame()

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = [f for f in zip_ref.namelist() if f.startswith('raw-img/') and f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    complete_df['file_name'] = pd.Series(file_list)
    complete_df['class_name'] = complete_df['file_name'].map(lambda x: x.split('/')[-2])

#### COMPROBACIÓN ####
print("Número total de imágenes:", len(complete_df))
class_counts = complete_df['class_name'].value_counts()
print(class_counts)

***Dividimos "complete_df" en entrenamiento y test :***

In [ ]:
complete_df = complete_df.sample(frac=1, random_state=42).reset_index(drop=True)

test_size = 0.2
train_df_list = []
test_df_list = []

classes = complete_df['class_name'].unique()

for class_name in classes:
    class_subset = complete_df[complete_df['class_name'] == class_name]
    
    test_count = int(len(class_subset) * test_size)
    
    test_df_list.append(class_subset.iloc[:test_count])
    train_df_list.append(class_subset.iloc[test_count:])

train_df = pd.concat(train_df_list).reset_index(drop=True)
test_df = pd.concat(test_df_list).reset_index(drop=True)

train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

#### COMPROBACIÓN ####
print(f"Nº de imágenes en train: {len(train_df)}")
print(train_df['class_name'].value_counts())
print()
print(f"Nº de imágenes en test: {len(test_df)}")
print(test_df['class_name'].value_counts())
print()
print("Ejm. conjunto train:")
print(train_df.head(10))
print()
print("Ejm. conjunto test:")
print(test_df.head(10))

***Imágenes de entrenamiento y test antes del pre-procesado :***

In [ ]:
def mostrar_imagenes_antes_preprocesado(df):
    plt.figure(figsize=(15, 15))
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        grouped = df.groupby('class_name')
        for i, (class_name, group) in enumerate(grouped):
            image_path = group['file_name'].iloc[0]
            with zip_ref.open(image_path) as image_file:
                image = Image.open(image_file).convert("RGB")
            plt.subplot(1, len(grouped), i + 1)
            plt.imshow(image)
            plt.title(class_name)
            plt.axis('off')
    plt.show()

mostrar_imagenes_antes_preprocesado(train_df)
mostrar_imagenes_antes_preprocesado(test_df)


***Pre-procesado de las imágenes :***

In [ ]:
IMG_SIZE = (227, 227)
X_train = []
y_train = []
X_test = []
y_test = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{device} disponible")

transform = transforms.Compose([transforms.Resize(IMG_SIZE), transforms.ToTensor()])

def preprocesado_imagen(zip_ref, image_path):
    with zip_ref.open(image_path) as image_file:
        image = Image.open(image_file).convert("RGB")
        image = transform(image)
        return image

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for index, row in train_df.iterrows():
        image_tensor = preprocesado_imagen(zip_ref, row['file_name'])
        X_train.append(image_tensor)
        y_train.append(row['class_name'])

    for index, row in test_df.iterrows():
        image_tensor = preprocesado_imagen(zip_ref, row['file_name'])
        X_test.append(image_tensor)
        y_test.append(row['class_name'])

X_train = torch.stack(X_train)
X_test = torch.stack(X_test)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

encoder = OneHotEncoder(sparse_output=False, categories='auto')
y_train = torch.tensor(y_train_encoded, dtype=torch.long)
y_test = torch.tensor(y_test_encoded, dtype=torch.long)

#### COMPROBACIÓN ####
print("X_train:", X_train.shape, X_train.device)
print("y_train:", y_train.shape, y_train.device)
print("X_test:", X_test.shape, X_test.device)
print("y_test:", y_test.shape, y_test.device)

***Añadir conjunto de validación :***

In [ ]:
batch_size = 32

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("X_train:", X_train.shape, X_train.device)
print("y_train:", y_train.shape, y_train.device)
print("X_val:", X_val.shape, X_val.device)
print("y_val:", y_val.shape, y_val.device)

***Datasets y DataLoaders :***

In [ ]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=12, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=12, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=12, pin_memory=True)

# ***AlexNet :***

In [ ]:
num_classes = 5

#Sin pesos preentrenados
alexnet_scratch = alexnet(weights=None)
alexnet_scratch.classifier[6] = nn.Linear(4096, num_classes)

#Con pesos preentrenados
alexnet_weights = AlexNet_Weights.DEFAULT
alexnet_pretrained = alexnet(weights=alexnet_weights)
alexnet_pretrained.classifier[6] = nn.Linear(4096, num_classes)

#Congelar capas
for param in alexnet_pretrained.parameters():
    param.requires_grad = False
for param in alexnet_pretrained.classifier[6].parameters():
    param.requires_grad = True

# ***ResNet50 :***

In [ ]:
num_classes = 5

#Sin pesos preentrenados
resnet50_scratch = resnet50(weights=None)
resnet50_scratch.fc = nn.Linear(resnet50_scratch.fc.in_features, num_classes)

#Con pesos preentrenados
resnet50_weights = ResNet50_Weights.DEFAULT
resnet50_pretrained = resnet50(weights=resnet50_weights)
resnet50_pretrained.fc = nn.Linear(resnet50_pretrained.fc.in_features, num_classes)

#Congelar capas
for param in resnet50_pretrained.parameters():
    param.requires_grad = False
for param in resnet50_pretrained.fc.parameters():
    param.requires_grad = True

# ***DenseNet121 :***

In [ ]:
num_classes = 5

#Sin pesos preentrenados
densenet121_scratch = densenet121(weights=None)
densenet121_scratch.classifier = nn.Linear(densenet121_scratch.classifier.in_features, num_classes)

#Con pesos preentrenados
densenet121_weights = DenseNet121_Weights.DEFAULT
densenet121_pretrained = densenet121(weights=densenet121_weights)
densenet121_pretrained.classifier = nn.Linear(densenet121_pretrained.classifier.in_features, num_classes)

#Congelar capas
for param in densenet121_pretrained.parameters():
    param.requires_grad = False
for param in densenet121_pretrained.classifier.parameters():
    param.requires_grad = True

# ***ConvNeXt Tiny :***

In [ ]:
num_classes = 5

#Sin pesos preentrenados
convnext_tiny_scratch = convnext_tiny(weights=None)
in_features = convnext_tiny_scratch.classifier[2].in_features
convnext_tiny_scratch.classifier[2] = nn.Linear(in_features, num_classes)

#Con pesos preentrenados
convnext_tiny_weights = ConvNeXt_Tiny_Weights.DEFAULT
convnext_tiny_pretrained = convnext_tiny(weights=convnext_tiny_weights)
in_features = convnext_tiny_pretrained.classifier[2].in_features
convnext_tiny_pretrained.classifier[2] = nn.Linear(in_features, num_classes)

#Congelar capas
for param in convnext_tiny_pretrained.parameters():
    param.requires_grad = False
for param in convnext_tiny_pretrained.classifier[2].parameters():
    param.requires_grad = True


***Inicialización del modelo :***

In [ ]:
num_epochs = 25
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### AlexNet ####
#modelo = alexnet_scratch.to(device)
#modelo = alexnet_pretrained.to(device)

#### ResNet50 ####
#modelo = resnet50_scratch.to(device)
#modelo = resnet50_pretrained.to(device)

#### DenseNet121 ####
#modelo = densenet121_scratch.to(device)
#modelo = densenet121_pretrained.to(device)

#### ConvNeXt Tiny ####
#modelo = convnext_tiny_scratch.to(device)
modelo = convnext_tiny_pretrained.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, modelo.parameters()), lr=0.001)


***Entrenamiento del modelo :***

In [ ]:
train_accuracies = []
train_losses = []
val_accuracies = []
val_losses = []

for epoch in range(num_epochs):

    #Train
    modelo.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = modelo(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        _, predicted = torch.max(output, 1)
        actual = labels
        total_train += labels.size(0)
        correct_train += (predicted == actual).sum().item()

        progress_bar.set_postfix(loss=total_loss / (progress_bar.n + 1))

    avg_train_loss = total_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train

    #Val
    modelo.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = modelo(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            actual = labels
            total_val += labels.size(0)
            correct_val += (predicted == actual).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * correct_val / total_val

    train_accuracies.append(train_acc)
    train_losses.append(avg_train_loss)
    val_accuracies.append(val_acc)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} ✅ | Train Acc: {train_acc:.2f}% - Train Loss: {avg_train_loss:.4f} | Val Acc: {val_acc:.2f}% - Val Loss: {avg_val_loss:.4f}")

# ***Resultados AlexNet :***

***Representación 2 :***

1. ***Longitud 8 :***  

2. ***Longitud 18 :***   

3. ***Longitud 28 :***

***Representación 3 :***

1. ***Longitud 8 :***  

2. ***Longitud 18 :***   

3. ***Longitud 28 :***

---

# ***Resultados ResNet50 :***

***Representación 2 :***

1. ***Longitud 8 :***  

2. ***Longitud 18 :***   

3. ***Longitud 28 :***

***Representación 3 :***

1. ***Longitud 8 :***  

2. ***Longitud 18 :***   

3. ***Longitud 28 :***

---

# ***Resultados DenseNet121 :***

***Representación 2 :***

1. ***Longitud 8 :***  

2. ***Longitud 18 :***   

3. ***Longitud 28 :***

***Representación 3 :***

1. ***Longitud 8 :***  

2. ***Longitud 18 :***   

3. ***Longitud 28 :***

---

# ***Resultados ConvNeXt Tiny :***

***Representación 2 :***

1. ***Longitud 8 :***  

2. ***Longitud 18 :***   

3. ***Longitud 28 :***

***Representación 3 :***

1. ***Longitud 8 :***  

2. ***Longitud 18 :***   

3. ***Longitud 28 :*** 

***Graficar los resultados del entrenamiento :***

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_accuracies, label="Train Accuracy")
plt.plot(range(1, num_epochs + 1), val_accuracies, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_losses, label="Train Loss")
plt.plot(range(1, num_epochs + 1), val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()

output_dir = './Resultados_entrenamiento/Representacion_2/'
os.makedirs(output_dir, exist_ok=True)
plt.savefig(f'{output_dir}ResNet50_8_con.png')
plt.show()